In [32]:
using DataFrames, PyPlot, ScikitLearn, LowRankModels, CSV, LinearAlgebra

In [14]:
include("proxgrad.jl")


proxgrad_const

In [10]:
taylor = CSV.read("taylor.csv")


,Column1,danceability,energy,key,loudness,mode,speechiness,acousticness
,Int64,Float64,Float64,Int64,Float64,Int64,Float64,Float64
1,0,0.588,0.825,4,-5.882,1,0.0328,0.000197
2,1,0.602,0.896,1,-4.267,0,0.0437,0.0773
3,2,0.705,0.621,2,-8.086,1,0.0334,0.101
4,3,0.622,0.469,3,-6.798,0,0.0363,0.00454
5,4,0.602,0.609,0,-7.803,1,0.0243,0.0382
6,5,0.658,0.729,7,-6.561,1,0.0378,0.00215
7,6,0.567,0.481,4,-7.357,1,0.027,0.0173
8,7,0.628,0.676,7,-5.911,1,0.0916,0.00957
9,8,0.729,0.748,0,-6.67,1,0.0245,0.307


In [ ]:
 plot_ord_data(X=X,y=y; margin=.05)
    p=plot()
    for i=1:4
        leveli = vec(y.==i)
        scatter!(X[leveli,1],X[leveli,2], color=plotcolor[i], shape=plotshape[i], label="$i")
    end
    xlabel!(L"x_1")
    ylabel!(L"x_2")
    range_y = maximum(X[:,2]) - minimum(X[:,2])
    range_x = maximum(X[:,1]) - minimum(X[:,1])
    ylims!((minimum(X[:,2])-margin*range_y,maximum(X[:,2])+margin*range_y))
    xlims!((minimum(X[:,1])-margin*range_x,maximum(X[:,1])+margin*range_x))
    p
end

In [11]:
drake= CSV.read("drake.csv")

,Column1,danceability,energy,key,loudness,mode,speechiness,acousticness
,Int64,Float64,Float64,Int64,Float64,Int64,Float64,Float64
1,0,0.373,0.56,9,-7.615,0,0.0448,0.132
2,1,0.703,0.699,2,-7.322,1,0.044,0.00113
3,2,0.442,0.421,2,-11.455,1,0.137,0.376
4,3,0.581,0.673,7,-7.111,1,0.162,0.032
5,4,0.785,0.633,7,-8.223,1,0.169,0.0259
6,5,0.62,0.722,1,-7.022,1,0.322,0.0733
7,6,0.66,0.311,0,-8.681,1,0.113,0.625
8,7,0.872,0.407,1,-10.036,1,0.256,0.152
9,8,0.536,0.446,1,-10.489,1,0.162,0.104


In [26]:
# y = convert(Array{Float64,1}, drake[:,15])
# x = convert(Array{Float64,2}, drake[1:14])
# loss = OrdinalHingeLoss()

# λ = .1
# reg = λ*QuadReg()

# # minimize 1/n ||Xw - y||^2 + λ||w||^2
# #w = proxgrad(loss, reg, X, y, maxiters=5, c=.1, stepsize=1, max_inner_iters=10000) 
# w = proxgrad(loss, reg, x, y, stepsize=0.001, maxiters=500)

# # norm(x*w-y) / norm(y)
# print(x*w)

n=354
Xoffset = convert(Array{Float64,2}, drake[1:14])
y = convert(Array{Float64,1}, drake[:,15])

loss = OrdinalHingeLoss(1,4)
mul!(loss, 1/n)
w_ordhinge = proxgrad(loss, ZeroReg(), Xoffset, y, stepsize=1, maxiters=200)
@show w_ordhinge

# plot the fit
p=plot_ord_data()
plotline!(w_ordhinge[1:2], w_ordhinge[3] - 1.5, color=plotcolor[1], label="1 vs 2,3,4")
plotline!(w_ordhinge[1:2], w_ordhinge[3] - 2.5, color=plotcolor[2], label="1,2 vs 3,4")
plotline!(w_ordhinge[1:2], w_ordhinge[3] - 3.5, color=plotcolor[3], label="1,2,3 vs 4")
ntest = 200
Xtest = [5*rand(ntest,2) ones(ntest)]
plot_imputed!(loss, Xtest, reshape(w_ordhinge, (3,1)), plotcolor, plotshape)
# savefig("ord_hinge.pdf")

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[26]:16
└ @ Core In[26]:16


w_ordhinge = [966786.5621468886, 3003.194025423719, 2825.7676717514078, 22717.3050847458, -33923.06013559329, 2307.209039548027, 1005.107787853111, 1157.3507325480268, 40.75843914282482, 836.94945451977, 1840.2795265536795, 578286.2619293766, 1.1663037073502793e9, 9.43428918926558e6]


UndefVarError: UndefVarError: plot_ord_data not defined

In [27]:
# n=60
Xoffset =  convert(Array{Float64,2}, drake[1:14])
y = convert(Array{Float64,1}, drake[:,15])
loss = BvSLoss(100, bin_loss=LogisticLoss())
# mul!(loss, 1/n)
w_bvs = proxgrad(loss, ZeroReg(), Xoffset, convert(Array{Int64,1},y), stepsize=0.01, maxiters=5000)
@show w_bvs

w_bvs*Xoffset

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[27]:1
└ @ Core In[27]:1


w_bvs = [6.09484449679136e-6 6.413726423544181e-6 7.942922996284549e-6 7.4846355001272946e-6 6.8510187869031526e-6 5.1349040705334405e-6 3.904384206176202e-6 3.6397844142151026e-6 3.094638260467803e-6 2.8829394294043665e-6 3.442691655832438e-6 3.442691655832438e-6 3.2522798736805083e-6 3.350664329380583e-6 3.350664329380583e-6 3.35046898268752e-6 3.169616624354698e-6 2.9678190360928384e-6 2.9678190360928384e-6 2.9678190360928384e-6 3.1259443091101898e-6 3.1259443091101898e-6 3.1259443091101898e-6 3.1917042942905213e-6 3.2490887844531786e-6 3.2490887844531786e-6 3.185504182946686e-6 3.2674932321391748e-6 3.275891927607708e-6 3.2406970824032584e-6 3.2917347389555817e-6 3.319326921225312e-6 3.319326921225312e-6 3.319326921225312e-6 3.330146692686329e-6 3.330146692686329e-6 3.4491445649159997e-6 3.4491445649159997e-6 3.4491445649159997e-6 3.501380327568923e-6 3.3331927943523213e-6 3.264975636701669e-6 3.205096345091942e-6 3.205096345091942e-6 3.205096345091942e-6 3.5689858656013073e-6 3.79

DimensionMismatch: DimensionMismatch("A has dimensions (14,99) but B has dimensions (354,14)")